## WebScrapping

In [1]:
import sys
from pathlib import Path

from tqdm import tqdm

In [2]:
sys.path.append(str(Path('..').resolve()))

In [3]:
from thesis_search.utils.crawler import HSEVKRScrapper

In [4]:
thesis_pages = [
    'https://www.hse.ru/ba/ling/students/diplomas/',  # фикл
    'https://www.hse.ru/ba/se/students/diplomas/',  # пи
    'https://www.hse.ru/en/ba/ami/students/diplomas/',  # пми
]

In [5]:
scrapper = HSEVKRScrapper()

In [6]:
%%time
all_results = []
for page in thesis_pages:
    result = scrapper.crawl(page, pagelimit=50, sleep=0)
    all_results.extend(result)
len(all_results)

CPU times: user 6.63 s, sys: 1.59 s, total: 8.22 s
Wall time: 1min 16s


1500

In [7]:
all_results[-1]

Thesis(thesis_id=474831623, year=2021, title='Критерии модулярности и суммарной средней связи для выявления сообществ: экспериментальное сравнение', student='Куропаткина Лариса Дмитриевна', supervisors=['Миркин Борис Григорьевич'], learn_program='Прикладная математика и информатика', abstract='Иерархическая кластеризация широко используется в анализе данных. Несмотря на некоторые недостатки, большое достоинство этого метода заключается в построении иерархии кластеров. Чтобы использовать иерархическую кластеризацию надо определить критерий связи, согласно которому кластеры будут объединяться или делиться. В этой работе проведено экспериментальное сравнение нескольких таких критериев: полусредняя связь, модулярность, суммарная связь за вычетом среднего.', text_lemmatized=None, file=None)

## Добавление в базу данных

In [8]:
from thesis_search.utils.database import DBHandler
from thesis_search.utils.models import filter_texts

In [9]:
db_path = Path('../data/theses.db').resolve()
db_path

PosixPath('/Users/viktoriaknazkova/Desktop/me/study/4 курс/инф_поиск/project-vknyazkova/data/theses.db')

In [10]:
db = DBHandler(db_path)

In [11]:
all_results = filter_texts(all_results)

In [12]:
for result in tqdm(all_results):
    db.add_thesis_info(result)

100%|██████████████████████████████████████| 1484/1484 [00:01<00:00, 920.24it/s]


## Лемматизация

In [13]:
!pip install spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 1.3 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [14]:
from functools import partial

import pandas as pd
import spacy

from thesis_search.utils.models import preprocessing

In [15]:
fast_nlp = spacy.load("ru_core_news_sm", exclude=["ner"])

In [16]:
corpus = db.get_raw_texts()
df = pd.DataFrame(corpus, columns=['id', 'text'])

In [17]:
fast_preprocessing = partial(preprocessing, nlp=fast_nlp)

In [18]:
%%time
df['lemmatized'] = df.text.apply(fast_preprocessing)
df.head(1)

CPU times: user 43.8 s, sys: 243 ms, total: 44.1 s
Wall time: 44.1 s


,id,text,lemmatized
0,153006854,Работа посвящена разметке словаря для автомати...,работа посвятить разметка словарь автоматическ...


In [19]:
db.add_lemmatization(df['id'].to_list(), df['lemmatized'].to_list())